In [1]:
import requests
import time
import os
from dotenv import load_dotenv
import pyvista as pv
import numpy as np
import pandas as pd

In [2]:
load_dotenv()
URL_PHYPHOX = os.getenv('URL_PHYPHOX', 'http://localhost:8080')

TO_READ = [
    "gyroX", "gyroY", "gyroZ", "gyro_time", # gyroscope data
    "accX", "accY", "accZ", "acc_time", # accelerometer data
    "graX", "graY", "graZ", "graT",  # gravity data
    "lin_accX", "lin_accY", "lin_accZ", "lin_acc_time"  # linear acceleration data
]

url_get = f"{URL_PHYPHOX}/get?{'&'.join(TO_READ)}"  



In [3]:
measures_count = 0

measures = dict()
gravity = dict()
linear_acceleration = dict()
acceleration = dict()
gyroscope = dict()

In [ ]:
while True:

    response = requests.get(url_get, timeout=1)

    response.raise_for_status() 

    data = response.json()

    grax = data['buffer']['graX']['buffer'][0]
    gray = data['buffer']['graY']['buffer'][0]
    graz = data['buffer']['graZ']['buffer'][0]
    grat = data['buffer']['graT']['buffer'][0]

    print(f"Gravity: x={grax:.2f}, y={gray:.2f}, z={graz:.2f}, t={grat:.2f}")

    lin_accx = data['buffer']['lin_accX']['buffer'][0]
    lin_accy = data['buffer']['lin_accY']['buffer'][0]
    lin_accz = data['buffer']['lin_accZ']['buffer'][0]
    lin_acct = data['buffer']['lin_acc_time']['buffer'][0]

    print(f"Linear Acceleration: x={lin_accx:.2f}, y={lin_accy:.2f}, z={lin_accz:.2f}, t={lin_acct:.2f}")

    accx = data['buffer']['accX']['buffer'][0]
    accy = data['buffer']['accY']['buffer'][0]
    accz = data['buffer']['accZ']['buffer'][0]
    acct = data['buffer']['acc_time']['buffer'][0]

    print(f"Acceleration: x={accx:.2f}, y={accy:.2f}, z={accz:.2f}, t={acct:.2f}")

    gyrx = data['buffer']['gyroX']['buffer'][0]
    gyry = data['buffer']['gyroY']['buffer'][0]
    gyrz = data['buffer']['gyroZ']['buffer'][0]
    gyrt = data['buffer']['gyro_time']['buffer'][0]

    print(f"Gyroscope: x={gyrx:.2f}, y={gyry:.2f}, z={gyrz:.2f}, t={gyrt:.2f}")

    print("-" * 40)
    time.sleep(0.1)

    measures[measures_count] = {
        'gravity': (np.array([grax, gray, graz]), grat),
        'linear_acceleration': (np.array([lin_accx, lin_accy, lin_accz]), lin_acct),
        'acceleration': (np.array([accx, accy, accz]), acct),
        'gyroscope': (np.array([gyrx, gyry, gyrz]), gyrt)
    }

    gravity[measures_count] = {'x': grax, 'y': gray, 'z': graz, 't': grat}
    linear_acceleration[measures_count] = {'x': lin_accx, 'y': lin_accy, 'z':lin_accz, 't': lin_acct }
    acceleration[measures_count] = {'x': accx, 'y': accy, 'z': accz, 't': acct}
    gyroscope[measures_count] = {'x': gyrx, 'y': gyry, 'z': gyrz, 't': gyrt}
    
    measures_count += 1
    print(measures_count)
    if measures_count >= 1000:
        break

Gravity: x=-0.02, y=0.15, z=9.81, t=3.29
Linear Acceleration: x=0.01, y=0.00, z=0.02, t=3.29
Acceleration: x=-0.01, y=0.15, z=9.83, t=3.29
Gyroscope: x=-0.00, y=0.00, z=0.00, t=3.29
----------------------------------------
1
Gravity: x=-0.02, y=0.15, z=9.81, t=3.41
Linear Acceleration: x=-0.00, y=-0.00, z=0.03, t=3.41
Acceleration: x=-0.02, y=0.15, z=9.83, t=3.41
Gyroscope: x=-0.00, y=0.00, z=-0.00, t=3.41
----------------------------------------
2
Gravity: x=-0.02, y=0.15, z=9.81, t=3.55
Linear Acceleration: x=-0.01, y=-0.00, z=0.03, t=3.55
Acceleration: x=-0.03, y=0.15, z=9.84, t=3.54
Gyroscope: x=-0.00, y=-0.00, z=-0.00, t=3.55
----------------------------------------
3
Gravity: x=-0.02, y=0.15, z=9.81, t=3.67
Linear Acceleration: x=0.00, y=-0.00, z=0.03, t=3.67
Acceleration: x=-0.02, y=0.15, z=9.84, t=3.66
Gyroscope: x=0.00, y=0.00, z=0.00, t=3.67
----------------------------------------
4
Gravity: x=-0.02, y=0.15, z=9.81, t=3.79
Linear Acceleration: x=-0.01, y=-0.01, z=0.02, t=3.7

ConnectTimeout: HTTPConnectionPool(host='192.168.0.109', port=80): Max retries exceeded with url: /get?gyroX&gyroY&gyroZ&gyro_time&accX&accY&accZ&acc_time&graX&graY&graZ&graT&lin_accX&lin_accY&lin_accZ&lin_acc_time (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x00000195628CBDF0>, 'Connection to 192.168.0.109 timed out. (connect timeout=1)'))

In [5]:
list(measures.keys())[-1]

182

In [ ]:
df_measures = pd.DataFrame(measures)
df_gravity = pd.DataFrame(gravity)
df_linear_acceleration = pd.DataFrame(linear_acceleration)
df_acceleration = pd.DataFrame(acceleration)
df_gyroscope = pd.DataFrame(gyroscope)

In [15]:
df_gravity_transposed = df_gravity.transpose()
df_linear_acceleration_transposed = df_linear_acceleration.transpose()
df_acceleration_transposed = df_acceleration.transpose()
df_gyroscope_transposed = df_gyroscope.transpose()

In [24]:
df_gravity_transposed.to_csv(path_or_buf=r"data\base\pos1_gravity.csv")
df_linear_acceleration_transposed.to_csv(r"data\base\pos1_linear_acceleration.csv")
df_acceleration_transposed.to_csv(r"data\base\pos1_acceleration.csv")
df_gyroscope_transposed.to_csv(r"data\base\pos1_gyroscope.csv")